Zadanie dotyczy wykorzystania odległości edycyjnej.

1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.
2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco:
        *k*los (dodanie litery k)
        klo*c* (zamiana s->c)
3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:
        los - kloc
        Łódź - Lodz
        kwintesencja - quintessence
        ATGAATCTTACCGCCTCG - ATGAGGCTCTGGCCCCTG
       
4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.
5. Korzystając z gotowego tokenizera (np. spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.
6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.
7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów.
8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku.
9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

### Edit distance algorithm

In [1]:
import numpy as np
from enum import Enum

def delta(x, y):
    if x == y:
        return 0
    else:
        return 1
    
def edit_distance(x, y, d = delta):
    edit_table = np.empty((len(x) + 1, len(y) + 1))
    for i in range(len(x) + 1):
        edit_table[i,0] = i
    for i in range(len(y) + 1):
        edit_table[0,i] = i
    
    for i in range(len(x)):
        k = i + 1
        for j in range(len(y)):
            l = j + 1
            edit_table[k ,l] =  min(edit_table[k-1,  l] + 1, # Down
                                    edit_table[k,  l-1] + 1, # Right
                                    edit_table[k-1,l-1] + d(x[i],y[j])) # Diagonally 
            
    return edit_table

### Edit path and visualization

In [2]:
class Operation(Enum):
    INSERT = 0
    DELETE = 1
    CHANGE = 2
    NOCHANGE = 3
    
def get_edit_path(x, y, edit_table = None, d = delta):
    if edit_table == None:
        edit_table = edit_distance(x, y, d)
        
    operations = []
    i = len(x)
    j = len(y)
    
    while i > 0 or j > 0:
        if i > 0 and j > 0:
            # CHANGE
            if edit_table[i-1][j-1] <= min(edit_table[i][j-1], edit_table[i-1][j]):
                if edit_table[i-1][j-1] != edit_table[i][j]:
                    operations.append(Operation.CHANGE)
                else:
                    operations.append(Operation.NOCHANGE)
                i-=1
                j-=1
            # DELETE
            elif edit_table[i][j-1] < edit_table[i-1][j-1]:
                if edit_table[i][j-1] != edit_table[i][j]:
                    operations.append(Operation.DELETE)
                else:
                    operations.append(Operation.NOCHANGE)
                j-=1
            # INSERT
            else:
                if edit_table[i-1][j] != edit_table[i][j]:
                    operations.append(Operation.INSERT)
                else:
                    operations.append(Operation.NOCHANGE)
                i-=1
        # INSERT START
        elif i > 0:
            operations.append(Operation.INSERT)
            i-=1
        # DELETE START
        else:
            operations.append(Operation.DELETE)
            j-=1
    return operations[::-1]


def visualize_path(x, y, edit_path = None):
    if edit_path == None:
        edit_path = get_edit_path(x, y)
        
    max_len = len(edit_path)
    xi = 0
    dlt = 0 # counter of deleted chars from string
    
    print(y)
    
    for i in range(max_len):

        if edit_path[i] == Operation.NOCHANGE:
            if i+1 < len(y):
                print(f"{y[:i-dlt]}*{y[i-dlt]}*{y[i+1-dlt:]}\t- Brak zmiany")
            else:
                print(f"{y[:i-dlt]}*{y[i-dlt]}*\t- Brak zmiany")
            xi += 1
            
        elif edit_path[i] == Operation.DELETE:
            if i+1-dlt < len(y):
                print(f"{y[:i-dlt]}* *{y[i-dlt+1:]}\t- Usunięcie znaku {y[i-dlt]}")
                y = y[:i-dlt] + y[i+1-dlt:]
            else:
                print(f"{y[:i-dlt]}* * \t- Usunięcie znaku {y[i-dlt]}")
                y = y[:i-dlt]
            dlt+=1
            
        elif edit_path[i] == Operation.INSERT:
            if i+1 < len(y):
                print(f"{y[:i]}*{x[xi]}*{y[i:]}\t- Dodanie znaku {x[xi]}")
                y = y[:i] + x[xi] + y[i:]
            else:
                print(f"{y[:i]}*{x[xi]}* \t- Dodanie znaku {x[xi]}")
                y = y[:i] + x[xi]
            xi += 1
                
        else:
            if i+1 < len(y):
                print(f"{y[:i-dlt]}*{x[xi]}*{y[i+1-dlt:]}\t- Zamiana znaku {y[i-dlt]} na {x[xi]}")
                y = y[:i-dlt] + x[xi] + y[i+1-dlt:]
            else:
                print(f"{y[:i-dlt]}*{x[xi]}* \t- Zamiana znaku {y[i-dlt]} na {x[xi]}")
                y = y[:i-dlt] + x[xi]
            xi += 1
            
    print(y)

In [3]:
x = "wojtk"
y = "wajak"

edit_dist = edit_distance(x, y, delta)
edit_path = get_edit_path(x, y)
edit_path

[<Operation.NOCHANGE: 3>,
 <Operation.CHANGE: 2>,
 <Operation.NOCHANGE: 3>,
 <Operation.CHANGE: 2>,
 <Operation.NOCHANGE: 3>]

In [4]:
visualize_path(x,y, edit_path)

wajak
*w*ajak	- Brak zmiany
w*o*jak	- Zamiana znaku a na o
wo*j*ak	- Brak zmiany
woj*t*k	- Zamiana znaku a na t
wojt*k*	- Brak zmiany
wojtk


In [5]:
visualize_path("kloc", "los")

los
*k*los	- Dodanie znaku k
k*l*os	- Brak zmiany
kl*o*s	- Brak zmiany
klo*c* 	- Zamiana znaku s na c
kloc


In [6]:
visualize_path("Łódź", "Lodz")

Lodz
*Ł*odz	- Zamiana znaku L na Ł
Ł*ó*dz	- Zamiana znaku o na ó
Łó*d*z	- Brak zmiany
Łód*ź* 	- Zamiana znaku z na ź
Łódź


In [7]:
visualize_path("quintessence", "kwintesencja")

kwintesencja
*q*wintesencja	- Zamiana znaku k na q
q*u*intesencja	- Zamiana znaku w na u
qu*i*ntesencja	- Brak zmiany
qui*n*tesencja	- Brak zmiany
quin*t*esencja	- Brak zmiany
quint*e*sencja	- Brak zmiany
quinte*s*encja	- Brak zmiany
quintes*s*encja	- Dodanie znaku s
quintess*e*ncja	- Brak zmiany
quintesse*n*cja	- Brak zmiany
quintessen*c*ja	- Brak zmiany
quintessenc* *a	- Usunięcie znaku j
quintessenc*e* 	- Zamiana znaku a na e
quintessence


In [8]:
visualize_path("ATGAGGCTCTGGCCCCTG", "ATGAATCTTACCGCCTCG")

ATGAATCTTACCGCCTCG
*A*TGAATCTTACCGCCTCG	- Brak zmiany
A*T*GAATCTTACCGCCTCG	- Brak zmiany
AT*G*AATCTTACCGCCTCG	- Brak zmiany
ATG*A*ATCTTACCGCCTCG	- Brak zmiany
ATGA*G*TCTTACCGCCTCG	- Zamiana znaku A na G
ATGAG*G*CTTACCGCCTCG	- Zamiana znaku T na G
ATGAGG*C*TTACCGCCTCG	- Brak zmiany
ATGAGGC*T*TACCGCCTCG	- Brak zmiany
ATGAGGCT*C*TACCGCCTCG	- Dodanie znaku C
ATGAGGCTC*T*ACCGCCTCG	- Brak zmiany
ATGAGGCTCT*G*CCGCCTCG	- Zamiana znaku A na G
ATGAGGCTCTG*G*CGCCTCG	- Zamiana znaku C na G
ATGAGGCTCTGG*C*GCCTCG	- Brak zmiany
ATGAGGCTCTGGC*C*CCTCG	- Zamiana znaku G na C
ATGAGGCTCTGGCC*C*CTCG	- Brak zmiany
ATGAGGCTCTGGCCC*C*TCG	- Brak zmiany
ATGAGGCTCTGGCCCC*T*CG	- Brak zmiany
ATGAGGCTCTGGCCCCT* *G	- Usunięcie znaku C
ATGAGGCTCTGGCCCCT*G*	- Brak zmiany
ATGAGGCTCTGGCCCCTG


### Longest common subsequence

In [9]:
def delta_lcs(x, y):
    if x == y:
        return 0
    else:
        return 2
    
def lcs(x, y):
    edit_dist = (edit_distance(x, y, delta_lcs))[-1][-1]
    return (len(x) + len(y) - edit_dist) / 2.0

In [10]:
lcs("wojtk", "wajaka")

3.0

### Tokenizing file

In [11]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from numpy.random import uniform

def tokenize_file(filepath):
    text = None
    with open(filepath, "r", encoding = 'UTF-8') as file:
        text = file.read()

    tokenizer = Tokenizer(Polish().vocab)
    tokens = tokenizer(text)
    
    return tokens

def remove_tokens(filepath, prob):
    tokened_text = tokenize_file(filepath)
    text_token_len = 0;
    new_tokens = []

    for token in tokened_text:
        if uniform() >= prob:
            text_token_len += 1
            new_tokens.append(token)

    return str(new_tokens).replace(",", "").split('\n')

In [12]:
julia  = remove_tokens("romeo-i-julia-700.txt", 0)
julia1 = remove_tokens("romeo-i-julia-700.txt", 0.03)
julia2 = remove_tokens("romeo-i-julia-700.txt", 0.03)

In [13]:
print(len(julia))
print(len(julia1))
print(len(julia2))

701
676
686


In [14]:
lcs(julia1, julia2)

566.0

In [15]:
with open("julia1.txt", "w", encoding = 'UTF-8') as file:
    file.write("\n".join(julia1))
    
with open("julia2.txt", "w", encoding = 'UTF-8') as file:
    file.write("\n".join(julia2))

### Diff

In [16]:
def lcs_diff(x, y):
    dp = np.empty((len(x) + 1, len(y) + 1))
    for i in range(1, len(x) + 1):
        for j in range(1, len(y) + 1):
            if x[i - 1] == y[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
            else:
                dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
    return dp

In [17]:
print(lcs_diff(julia1, julia2)[-1][-1] == lcs(julia1, julia2))
lcs_diff(julia1, julia2)

True


array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   1.,   1., ...,   1.,   1.,   1.],
       [  0.,   1.,   2., ...,   2.,   2.,   2.],
       ...,
       [  0.,   1.,   2., ..., 564., 564., 564.],
       [  0.,   1.,   2., ..., 564., 565., 565.],
       [  0.,   1.,   2., ..., 564., 565., 566.]])

In [18]:
def load_file_as_tokens(filepath):
    return remove_tokens(filepath, 0)

In [19]:
def get_common_lines(lines1, lines2):
    lcs_table = lcs_diff(lines1, lines2)
    common = []
    
    i = len(lines1)
    j = len(lines2)
    while i > 0 and j > 0:
        if lines1[i - 1] == lines2[j - 1]:
            common.append(lines1[i - 1])
            i -= 1
            j -= 1
        elif lcs_table[i - 1][j] > lcs_table[i][j - 1]:
            i -= 1
        else:
            j -= 1
    
    common.reverse()
    return common

In [20]:
def diff(filepath1, filepath2):
    lines1 = load_file_as_tokens(filepath1)
    lines2 = load_file_as_tokens(filepath2)
    
    common_lines = get_common_lines(lines1, lines2)
    i = 0
    j = 0
    
    for line in common_lines:
        while (i < len(lines1) and lines1[i] != line) or (j < len(lines2) and lines2[j] != line):
            if i < len(lines1) and lines1[i] != line:
                print("< " + str(i) + lines1[i] + "\n")
                i += 1
            if j < len(lines2) and lines2[j] != line:
                print("> " + str(j) + lines2[j] + "\n")
                j += 1
        i += 1
        j += 1

    while i < len(lines1):
        print("< " + str(i) + lines1[i] + "\n")
        i += 1

    while j < len(lines2):
        print("> " + str(j) + lines2[j] + "\n")
        j += 1

In [21]:
diff("julia1.txt","julia2.txt")

< 11   * PARYS — młody Weroneńczyk szlachetnego rodu krewny księcia 

> 11   * PARYS — młody szlachetnego rodu krewny księcia 

< 12   * MONTEKI KAPULET — naczelnicy dwóch domów nieprzyjaznych 

> 12   * MONTEKI KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie 

< 19   * JAN — brat tegoż zgromadzenia 

> 19   * JAN — brat z zgromadzenia 

< 20   * BALTAZAR — służący Romea 

> 20   * BALTAZAR służący Romea 

< 23   * 

> 23   * APTEKARZ 

< 29   * PANI KAPULET małżonka Kapuleta 

> 29   PANI KAPULET — małżonka Kapuleta 

< 32   * Obywatele weroneńscy różne osoby płci obojej liczący się do przyjaciół obu domów maski straż wojskowa i inne osoby. 

> 32   * Obywatele weroneńscy różne osoby płci obojej liczący się do przyjaciół obu maski straż wojskowa i inne osoby. 

< 37  Rzecz odbywa się przez większą część sztuki w Weronie przez część piątego aktu w Mantui. 

> 37  Rzecz odbywa się przez większą część sztuki w Weronie przez piątego aktu w Mantui. 

< 46  Tam gdzie się ta rozgrywa w